In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
from os.path import join

import numpy as np
import pandas as pd

from scipy.spatial import distance_matrix

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

try:
    import dgl
except:
    !pip install dgl
    import dgl
    
import warnings
warnings.filterwarnings('ignore')

In [3]:
import torch.nn.functional as F

In [4]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
print('Number of categories:', dataset.num_classes)

In [5]:
g=dataset[0]
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats,o_feats ,h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, o_feats)
        self.conv2 = GraphConv(o_feats, h_feats)
        self.conv3 = GraphConv(h_feats, num_classes)
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.leaky_relu(h)
        h = self.conv2(g,h) 
        h = F.relu(h)
        h = self.conv3(g, h)
        return h

# Create the model with given dimensions
model = GCN(g.ndata['feat'].shape[1], 64, 16, dataset.num_classes)

In [6]:
test_score = []
losses = []
validation_score = []
train_score = []
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(500):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])
        losses.append(loss)
        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()
        #storing the metrics in lists format
        train_score.append(train_acc)
        validation_score.append(val_acc)
        test_score.append(test_acc)
        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 20 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))
model = GCN(g.ndata['feat'].shape[1], 64 ,16, dataset.num_classes)
train(g, model)